In [ ]:
# Data Collection

import nltk
# nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

In [ ]:
# load the text data from the Gutenberg corpus
data = gutenberg.raw('shakespeare-hamlet.txt')

# save the data to a text file
with open('hamlet.txt', 'w') as f:
    f.write(data)

In [ ]:
# Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# load the data

with open('hamlet.txt','r') as f:
    text = f.read().lower()

# tokenize the text
# here we are segregate the each word from the corpus of text given and assign a unique integer to each word

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

In [ ]:
# create input sequences and corresponding labels
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0] # convert each line to a sequence of integers
    # [0] is used becoz it stores the result all the texts in a list within a list
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]  
        input_sequences.append(n_gram_sequence)

In [ ]:
# Padding Sequences
max_sequence_len = max(len(x) for x in input_sequences)

input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [ ]:
# create predictors and label
x=input_sequences[:,:-1]
y=input_sequences[:,-1]


In [ ]:
# convert y to categorical 

y=to_categorical(y,num_classes=total_words)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [24]:
# LSTM Model Building

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

# Define the model
model = Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
    )



d:\Generative AI\Projects\LSTM\venv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

In [26]:
# training the model

history = model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=50,
    verbose=1,
    callbacks=[early_stop]
)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.0330 - loss: 6.8946 - val_accuracy: 0.0336 - val_loss: 6.6787
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0391 - loss: 6.4668 - val_accuracy: 0.0398 - val_loss: 6.7588
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0456 - loss: 6.3294 - val_accuracy: 0.0486 - val_loss: 6.8053
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.0530 - loss: 6.1831 - val_accuracy: 0.0534 - val_loss: 6.8110


In [30]:
# Prediction Function

def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len-1:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=0)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [32]:
input_text = "to be or not to be"
print(f"Input Text: {input_text}")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f"Predicted Next Word: {next_word}")

Input Text: to be or not to be
Predicted Next Word: the


In [35]:
# save the model
model.save('next_word_prediction_model_lstm.h5')
model.save('next_word_prediction_model_lstm.keras')

In [34]:
# Save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)